In [7]:
import pandas as pd

def harmonise_colonne_df(df: pd.DataFrame, column: str, mapping: dict) -> pd.DataFrame:
    """
    Harmonise les valeurs d'une colonne selon un dictionnaire de correspondance.

    Arguments :
        df      : DataFrame
        column  : nom de la colonne à harmoniser
        mapping : dictionnaire {valeur à remplacer: valeur cible}

    Retour :
        df modifié avec la colonne harmonisée
        
    exemple d'utilisation :
        mapping_sexe = {1: 'Homme', 2: 'Femme', 9: 'Inconnu'}
        df = harmonise_colonne_df(df, 'sexe', mapping_sexe)
    """
    if column not in df.columns:
        raise ValueError(f"La colonne '{column}' n'existe pas dans le DataFrame.")
    
    # Remplacement selon le dictionnaire
    df[column] = df[column].replace(mapping)
    
    return df


In [ ]:
def remplacer_valeurs_null(df):
    """
    Remplace automatiquement les valeurs nulles d'un DataFrame selon le type de chaque colonne.
    
    Règles :
      - colonnes numériques → remplacées par la moyenne
      - colonnes texte → remplacées par 'Inconnu'
      - colonnes datetime → remplacées par la date la plus ancienne du DF ou '1970-01-01' si vide
      - colonnes booléennes → remplacées par False

    Args:
        df (pd.DataFrame): le DataFrame à nettoyer
    
    Returns:
        pd.DataFrame: DataFrame avec les valeurs nulles remplacées
    """
    df = df.copy()

    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(df[col].mean())
        
        elif pd.api.types.is_string_dtype(df[col]) or df[col].dtype == 'object':
            df[col] = df[col].fillna('Inconnu')
        
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            if df[col].notna().any():
                min_date = df[col].min()
            else:
                min_date = pd.Timestamp('1970-01-01')
            df[col] = df[col].fillna(min_date)
        
        elif pd.api.types.is_bool_dtype(df[col]):
            df[col] = df[col].fillna(False)
        
        else:
            df[col] = df[col].fillna('Inconnu')
    return df


In [9]:
def remplacer_valeur(df, colonne, ancienne_valeur, nouvelle_valeur):
    """
    Remplace une valeur précise dans une colonne d'un DataFrame.
    
    Args:
        df (pd.DataFrame): le DataFrame à modifier
        colonne (str): le nom de la colonne à traiter
        ancienne_valeur (any): la valeur à remplacer
        nouvelle_valeur (any): la nouvelle valeur
        
    Returns:
        pd.DataFrame: le DataFrame modifié
    """
    df = df.copy()
    df[colonne] = df[colonne].replace(ancienne_valeur, nouvelle_valeur)
    return df


In [ ]:

def supprimer_colonnes(df, colonnes_a_supprimer):
    """
    Supprime une ou plusieurs colonnes d'un DataFrame.
    
    Args:
        df (pd.DataFrame): le DataFrame à modifier
        colonnes_a_supprimer (list): liste des noms de colonnes à supprimer
        
    Returns:
        pd.DataFrame: DataFrame avec les colonnes supprimées
    """
    df = df.copy()
    df.drop(columns=colonnes_a_supprimer, inplace=True)
    return df
